# Mapping GDELT Events

## Requirements

In [ ]:
!pip install --user gdelt

## Import modules

In [23]:
from datetime import date
from gdelt import gdelt as gdelt_client

## Getting GDELT events of today
Date must be formatted as a string.

In [29]:
def get_events(date):
    client = gdelt_client(version=2)
    events = client.Search(date.strftime("%Y %m %d"), table="events", coverage=True)
    del client
    return events

def get_today_events():
    return get_events(date.today())

In [30]:
get_today_events()

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,910963876,20190309,201903,2019,2019.1890,NaN,NaN,NaN,NaN,NaN,...,4,"Sydney, New South Wales, Australia",AS,AS02,154637,-33.883300,151.217000,-1603135,20200308001500,https://www.pancan.org/news/inspiring-women-le...
1,910963877,20190309,201903,2019,2019.1890,NaN,NaN,NaN,NaN,NaN,...,3,"Miami, Florida, United States",US,USFL,FL086,25.774300,-80.193700,295004,20200308001500,https://www.pancan.org/news/inspiring-women-le...
2,910963878,20190309,201903,2019,2019.1890,AFG,AFGHANISTAN,AFG,NaN,NaN,...,1,Afghanistan,AF,AF,NaN,33.000000,66.000000,AF,20200308001500,https://www.woad.com/syndicated-article/?id=11...
3,910963879,20190309,201903,2019,2019.1890,BRA,BRAZIL,BRA,NaN,NaN,...,4,"Rio De Janeiro, Estado do Rio, Brazil",BR,BR21,9961,-22.900000,-43.233300,-666610,20200308001500,https://www.clickondetroit.com/news/politics/2...
4,910963880,20190309,201903,2019,2019.1890,BRA,BRAZIL,BRA,NaN,NaN,...,3,"Palm Beach, Florida, United States",US,USFL,NaN,26.705600,-80.036400,288390,20200308001500,https://www.clickondetroit.com/news/politics/2...
5,910963881,20190309,201903,2019,2019.1890,GOVCOP,INSPECTOR GENERAL,NaN,NaN,NaN,...,3,"Washington, District of Columbia, United States",US,USDC,DC001,38.895100,-77.036400,531871,20200308001500,https://www.timesunion.com/news/article/Barr-i...
6,910963882,20190309,201903,2019,2019.1890,GOVCOP,INSPECTOR GENERAL,NaN,NaN,NaN,...,1,Russia,RS,RS,NaN,60.000000,100.000000,RS,20200308001500,https://www.timesunion.com/news/article/Barr-i...
7,910963883,20190309,201903,2019,2019.1890,JUD,PROSECUTOR,NaN,NaN,NaN,...,1,Afghanistan,AF,AF,NaN,33.000000,66.000000,AF,20200308001500,https://www.woad.com/syndicated-article/?id=11...
8,910963884,20190309,201903,2019,2019.1890,JUD,PROSECUTOR,NaN,NaN,NaN,...,1,Afghanistan,AF,AF,NaN,33.000000,66.000000,AF,20200308001500,https://www.woad.com/syndicated-article/?id=11...
9,910963885,20190309,201903,2019,2019.1890,TUR,TURKISH,TUR,NaN,NaN,...,1,Turkey,TU,TU,NaN,39.059012,34.911546,TU,20200308001500,https://www.click2houston.com/news/world/2020/...
